In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import avg,sum,col,array,array_contains,count
from pyspark.sql import types
import random

In [2]:
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('HW4').getOrCreate()

In [3]:
#Q1
review_rdd = sc.textFile('FileStore/tables/review.csv').map(lambda x: tuple(x.split('::')))
review_rdd = review_rdd.map(lambda x: (x[2],(float(x[3]),1))) \
                       .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
                       .map(lambda x: (x[0],x[1][0],x[1][1])) \
                       .map(lambda x: (x[0],x[1]/x[2],x[2]))

In [4]:
#review_id, user_id, business_id, stars
review_df = spark.createDataFrame(review_rdd.collect(),['business_id','AvgRating','NumOfReviews'])
print(review_df)

DataFrame[business_id: string, AvgRating: double, NumOfReviews: bigint]

In [5]:
print("Q1:")
review_df.show(n=10)

Q1:
+--------------------+------------------+------------+
 business_id| AvgRating|NumOfReviews|
+--------------------+------------------+------------+
qw5gR8vW7mSOK4VRO...| 4.105042016806722| 238|
Hnz1_h_D1eHSRtQqH...|3.0454545454545454| 66|
ZRm8fSEBn8DsSLD4o...|3.5757575757575757| 132|
Q0t7EEaygJ2hb4uFn...|3.4838709677419355| 62|
fcdjnsgO8Z5LthXUx...| 3.320388349514563| 103|
jTPr0dFk2JgGYdDiI...|3.6129032258064515| 93|
V9DXseGJNzLeCxUpM...|3.6666666666666665| 21|
piXuRfZ81xFGA64WF...| 4.38078967350038| 2634|
YWSsD10TzzqE1htRG...|2.0789473684210527| 38|
t6qBeLsoKOUmUwJCT...|3.8219178082191783| 73|
+--------------------+------------------+------------+
only showing top 10 rows

In [6]:
#Q2
business_rdd = sc.textFile('FileStore/tables/business.csv').map(lambda x: tuple(x.split('::')))

import re
list_re = re.compile(r'List\((.*)\)')

def make_list(string):
  parsed = list_re.match(string).groups()[0]
  split = parsed.split(',')
  stripped = list(map(lambda x: x.strip(), split))
  
  return stripped

def parse_state(string):
  address_split = string.split(',')
  state_split = address_split[-1].strip().split(' ')
  
  return state_split[0]

print(make_list('List(this, is, a)'))

business_df = spark.createDataFrame(business_rdd.collect(),['business_id','State','Categories'])

parse_state = functions.udf(parse_state)
make_list = functions.udf(make_list)

business_df = business_df.select('business_id', parse_state('State').alias('State'), make_list('Categories').alias('Categories'))

business_df = business_df.join(review_df,'business_id')

business_df_temp = business_df.select(['State','AvgRating','NumOfReviews'])

business_df_temp = business_df_temp.groupBy('State') \
                          .agg(sum('NumOfReviews').alias('NumOfReviews'), avg('AvgRating').alias('AvgRating')) \
                          .sort(col('AvgRating').desc())

print("Q2:")
business_df_temp.show(n=100)

['this', 'is', 'a']
Q2:
+-----+------------+------------------+
State|NumOfReviews| AvgRating|
+-----+------------+------------------+
 NC| 7623|3.8501092073353886|
 IN| 2829|3.7171534420367394|
 TX| 22482| 3.703492624926414|
 CA| 130182|3.6905629766795807|
 ON| 956| 3.689520596708223|
 MI| 10913| 3.656240435744404|
 RI| 10332|3.6367648970592517|
 WA| 16816| 3.633873539755566|
 MA| 51380| 3.594981538852904|
 GA| 14496|3.5818983182211097|
 VA| 1436| 3.551587631553725|
 PA| 18887|3.5442421521491396|
 IL| 6763|3.5008741861688217|
 NY| 22187| 3.487114710906497|
 MD| 5452| 3.371332620885174|
 NJ| 7337|3.3611607073099004|
+-----+------------+------------------+

In [7]:
#Q3
review_rdd = sc.textFile('FileStore/tables/review.csv').map(lambda x: tuple(x.split('::')))
review_df = spark.createDataFrame(review_rdd.collect(),['review_id','user_id','business_id','stars'])
review_df = review_df.select('review_id','user_id','business_id',review_df['stars'].cast('float').alias('stars'))

user_rdd = sc.textFile('FileStore/tables/user.csv').map(lambda x: tuple(x.split('::')))
user_df = spark.createDataFrame(user_rdd.collect(),['user_id','name','url'])

Q3_df = review_df.join(user_df,'user_id')

Q3_df = Q3_df.join(business_df,'business_id')

Q3_df.groupBy('user_id') \
     .agg(avg('stars'))

Out[43]: DataFrame[user_id: string, avg(stars): double]

In [8]:


def in_list(x):
  if "Colleges & Universities" in x:
    return True
  return False

in_list = functions.udf(in_list)

Q3_df_temp = Q3_df.select('user_id','name','AvgRating','Categories')
Q3_df_temp = Q3_df_temp.filter(in_list('Categories')==True)
Q3_df_temp = Q3_df_temp.select('user_id','name','AvgRating')

In [9]:
print('Q3:')
Q3_df_temp.show(n=10)

Q3:
+--------------------+----------+-----------------+
 user_id| name| AvgRating|
+--------------------+----------+-----------------+
TLzwLBhPoO33w85VO...| Nobu K.|4.647058823529412|
-f6amuvfIVS7VX65B...|Bridget B.|4.647058823529412|
VZ6oPvTXTsoQHuMfp...| Mary C.|4.647058823529412|
DrTmpnMmhxDs2wV7i...| Ashley M.|4.647058823529412|
k6VEe4G7dXWhooqFI...| Aj P.|4.647058823529412|
qo5zIoMosJhdb8F3-...| Krys A.|4.647058823529412|
q0UZnb1i90xQ2lvvA...| T T.|4.647058823529412|
fV63MN6UMMTCtJstL...| Greg C.|4.647058823529412|
nl8gWLDo6U6MjqzbB...| Vivian C.|4.647058823529412|
iWIUku8b5JZ_EhXBS...| Anne G.|4.647058823529412|
+--------------------+----------+-----------------+
only showing top 10 rows

In [10]:
#Q4
business_rdd = sc.textFile('FileStore/tables/business.csv').map(lambda x: tuple(x.split('::')))
review_rdd = sc.textFile('FileStore/tables/review.csv').map(lambda x: tuple(x.split('::')))
review_rdd = review_rdd.map(lambda x: (x[2],(float(x[3]),1))) \
                       .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
                       .map(lambda x: (x[0],x[1][0],x[1][1])) \
                       .map(lambda x: (x[0],x[1]/x[2],x[2]))
review_df = spark.createDataFrame(review_rdd.collect(),['business_id','AvgRating','NumOfReviews'])

review_df = review_df.select('business_id','AvgRating','NumOfReviews')

import re
list_re = re.compile(r'List\((.*)\)')

def make_list(string):
  parsed = list_re.match(string).groups()[0]
  split = parsed.split(',')
  stripped = list(map(lambda x: x.strip(), split))
  
  return stripped

def parse_state(string):
  address_split = string.split(',')
  state_split = address_split[-1].strip().split(' ')
  
  return state_split[0]

print(make_list('List(this, is, a)'))

business_df = spark.createDataFrame(business_rdd.collect(),['business_id','Address','Categories'])

parse_state = functions.udf(parse_state)
make_list = functions.udf(make_list)

business_df = business_df.select('business_id', 'Address', parse_state('Address').alias('State'), make_list('Categories').alias('Categories'))

business_df = business_df.join(review_df,'business_id')
business_df_temp = business_df.where('\'State\'==\'NY\'')

business_df_temp = business_df.select(['State','NumOfReviews','business_id','Address','Categories','AvgRating'])

def return_string(x):
  pass

avg_rating_df = business_df_temp.groupBy('Address') \
                          .agg(sum('NumOfReviews').alias('NumOfReviews'), avg('AvgRating').alias('AvgRating')) \
                          .sort(col('AvgRating').desc())

business_df_temp = business_df_temp.select(['business_id','Address','Categories']).join(avg_rating_df,'Address')

['this', 'is', 'a']

In [11]:
business_df_temp = business_df_temp.select(['business_id','Address','Categories','AvgRating']).sort(col('AvgRating').desc())
business_df_temp.show(n=10)

+--------------------+--------------------+--------------------+---------+
 business_id| Address| Categories|AvgRating|
+--------------------+--------------------+--------------------+---------+
WhvnlLZ6Ri57vJvJc...|HarlemHarlem, NY ...|[Hotels & Travel,...| 5.0|
6l9E4LzbW4Rq9OgQ3...|904 High StPalo A...|[Tires, Automotiv...| 5.0|
cNhZopm9zmuvR2SFF...|904 High StPalo A...|[Auto Repair, Aut...| 5.0|
5vcgEM1R2qUWfR9Va...|2001 GuadalupeSui...|[Tattoo, Beauty a...| 5.0|
-Tiy3NmLikBRA5NgM...|2826 Rio Grande S...|[Hair Salons, Bea...| 5.0|
ys0oidKmKNo6j3Dh_...|960 E Green StSte...|[Doctors, Health ...| 5.0|
WlLnJrLYn5h7bAw4y...|689 W. Foothill B...|[Skin Care, Day S...| 5.0|
fG-UbYw0DkImsirfA...|774 Marsh StSte D...|[Arts & Entertain...| 5.0|
HqTWm-plC4P_2H-Jn...|123 Main StColleg...|[Arts & Entertain...| 5.0|
uKAJl6XTiqmaV9Cg2...|689 11th Street N...|[Local Services, ...| 5.0|
+--------------------+--------------------+--------------------+---------+
only showing top 10 rows

In [12]:
#Q5

#group on user id agg count on states
Q5_df = Q3_df.groupBy('user_id')\
     .agg(avg('AvgRating').alias('AvgRating'),count('State').alias('StateCount'))


Q5_df = Q5_df.where(Q5_df['StateCount']>1)
          

#join  with og df
Q5_df = Q5_df.join(user_df,'user_id').select('user_id','name','AvgRating')
Q5_df.show(n=10)

+--------------------+----------+------------------+
 user_id| name| AvgRating|
+--------------------+----------+------------------+
-1KKYzibGPyUX-Mwk...| Nickie P.|3.8779343382267335|
-GKEFg_92pp0q842c...| Jeff M.| 3.673694686773835|
-M5t5azGaCThizWr1...| Kelsi P.|3.8418741853604996|
-NnADpikTs6IWM9tb...| Carrie T.|3.3764617239300785|
-hSLnGKUJtshbOw7C...| Brandy A.|3.6741702528721367|
-hgeKMHXU5S3HWDUn...|Tiffany S.|3.4995066296638915|
-lJmB5_6S3WlfUan5...| Olivia D.| 2.711135477582846|
-sy7JKUsUONSPD2ER...| Jus A.| 2.612755461592671|
0AlnetQ_-a4f7kY0S...| Susan C.|3.4667599789062558|
0EwPOQQOFkmkRV07z...| Tammy K.| 3.639511748098635|
+--------------------+----------+------------------+
only showing top 10 rows